In [1]:
import azureml.core

from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Environment, Datastore, Experiment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute import ComputeTarget, RemoteCompute, AmlCompute
from azureml.core.runconfig import RunConfiguration

from azureml.exceptions import WebserviceException
from azureml.data.data_reference import DataReference

from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData, Pipeline
import json
import os


In [2]:
with open('config.json', 'r') as jsonfile:
    ws_config = json.load(jsonfile)

interactive_auth = InteractiveLoginAuthentication(tenant_id=ws_config['tenantId'])

ws = Workspace(
    subscription_id=ws_config['subscription_id'],
    resource_group=ws_config['resource_group'],
    workspace_name=ws_config['workspace_name'],
    auth=interactive_auth,
)

In [3]:
blob_datastore_name='shiftdatastore' # Name of the datastore to workspace
container_name=os.getenv("BLOB_CONTAINER", "news20container") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "shiftreference") # Storage account name
account_key=os.getenv("AZURE_STORAGE_KEY") # Storage account key

try:
    datastore = Datastore.get(ws, blob_datastore_name)
except:
    datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                             datastore_name=blob_datastore_name, 
                                                             container_name=container_name, 
                                                             account_name=account_name,
                                                             account_key=account_key)


In [4]:
blob_input_data = DataReference(
    datastore,
    data_reference_name="rawdata",
    path_on_datastore="rawdata",
).as_download()

# Preprocessed files saved here
corpus_output_data = PipelineData(
    "corpus",
    datastore=datastore,
    output_path_on_compute="corpus",
    output_mode="upload"
)


# Create the compute config 
compute_name = "attach-dsvm"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    attach_config = RemoteCompute.attach_configuration(address = "127.0.0.1",
                                                     ssh_port=22,
                                                     username='compute',
                                                     password="compute")

    # If you authenticate with SSH keys instead, use this code:
    #                                                  ssh_port=22,
    #                                                  username='<username>',
    #                                                  password=None,
    #                                                  private_key_file="<path-to-file>",
    #                                                  private_key_passphrase="<passphrase>")
    
    # Attach the compute
    compute_target = ComputeTarget.attach(ws, compute_name, attach_config)



In [5]:
env = Environment.from_pip_requirements("sbsdeployment", "./requirements.txt")
runconfig = RunConfiguration()
runconfig.environment.docker.enabled = True

runconfig.environment = env


In [6]:

process_arguments = ["--input", blob_input_data, "--output", corpus_output_data]
process_step = PythonScriptStep(
    script_name="build_corpus.py",
    arguments=process_arguments,
    inputs=[blob_input_data],
    outputs=[corpus_output_data],
    compute_target=compute_target,
    source_directory=os.getcwd(),
    runconfig=runconfig,
    allow_reuse=True,
)

In [7]:
predictions = Pipeline(ws, steps=[process_step])
predictions_run = Experiment(ws, "build_corpus").submit(predictions)
predictions_run.wait_for_completion()

Created step build_corpus.py [69006bd5][b965e35b-45d0-471b-a63e-58c523bc5d47], (This step will run and generate new outputs)
Using data reference rawdata for StepId [aca33bb6][395508bc-299b-4d65-a244-73b2834045d8], (Consumers of this data are eligible to reuse prior runs.)
Submitted PipelineRun cce918c9-26ec-47bc-a7d9-af26717ad68e
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/build_corpus/runs/cce918c9-26ec-47bc-a7d9-af26717ad68e?wsid=/subscriptions/9017d57d-c4df-480d-b92d-7aea2266b0f0/resourcegroups/BAA_Shift/workspaces/News20Workspace
PipelineRunId: cce918c9-26ec-47bc-a7d9-af26717ad68e
Link to Portal: https://ml.azure.com/experiments/build_corpus/runs/cce918c9-26ec-47bc-a7d9-af26717ad68e?wsid=/subscriptions/9017d57d-c4df-480d-b92d-7aea2266b0f0/resourcegroups/BAA_Shift/workspaces/News20Workspace
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: ba2046f4-accb-4f5a-94fc-89e48e1a5926
Link to Portal: https://ml.azure.com/experiments/build_co

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Compute target attach-dsvm is in state Failed and thus not ready for runs.",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "e592dd1b1aa0bd76"
    },
    "environment": "northeurope",
    "location": "northeurope",
    "time": "2020-03-06T12:28:07.251875Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Compute target attach-dsvm is in state Failed and thus not ready for runs.\",\n        \"details\": []\n    },\n    \"correlation\": {\n        \"operation\": null,\n        \"request\": \"e592dd1b1aa0bd76\"\n    },\n    \"environment\": \"northeurope\",\n    \"location\": \"northeurope\",\n    \"time\": \"2020-03-06T12:28:07.251875Z\"\n}"
    }
}

In [ ]:

print('done')

In [ ]:


"error": {
        "code": "UserError",
        "message": "Compute target attach-dsvm is in state Failed and thus not ready for runs.",
        "details": []
    },